In [27]:
import cv2

if hasattr(cv2, "aruco"):
    print("Aruco is available.")


Aruco is available.


In [ ]:
import cv2
import numpy as np

VIDEO_IN = "aruco.mp4"
VIDEO_OUT = "aruco_green_overlay.mp4"

DICT = cv2.aruco.DICT_4X4_50
EXPECTED_IDS = [0, 1, 2, 3]  # TL-TR-BR-BL by your generator
MARGIN_PX = 10  # same as generator
ALPHA = 0.30  # overlay transparency

# ───────────── video info ─────────────
cap = cv2.VideoCapture(VIDEO_IN)
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
N = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

# ─────────── detector ───────────
dictionary = cv2.aruco.getPredefinedDictionary(DICT)
detector = cv2.aruco.ArucoDetector(dictionary, cv2.aruco.DetectorParameters())


# helper to shift a marker-corner by the known margin
def shift(pt, dx, dy):
    return np.array([pt[0] + dx, pt[1] + dy], dtype=np.float32)


# 1) pass: collect raw corner measurements  (NaN for “missing”)
tracks = np.full((N, 4, 2), np.nan, dtype=np.float32)  # frame × corner × xy
corner_map = {  # which marker-corner belongs to which screen corner
    0: (0, (-MARGIN_PX, -MARGIN_PX)),  # TL  from marker-corner 0
    1: (1, (MARGIN_PX, -MARGIN_PX)),  # TR  from marker-corner 1
    2: (2, (MARGIN_PX, MARGIN_PX)),  # BR  from marker-corner 2
    3: (3, (-MARGIN_PX, MARGIN_PX)),  # BL  from marker-corner 3
}

cap = cv2.VideoCapture(VIDEO_IN)
for f in range(N):
    ok, frame = cap.read()
    if not ok:
        break

    corners, ids, _ = detector.detectMarkers(frame)
    if ids is None:
        continue

    for idx, mid in enumerate(ids.flatten()):
        if mid in corner_map:
            c_idx, (dx, dy) = corner_map[mid]
            pt = shift(corners[idx][0][c_idx], dx, dy)  # use the matching marker-corner
            tracks[f, c_idx] = pt
cap.release()

# 1½) interpolate each corner track (x and y separately)
for corner in range(4):
    for dim in range(2):  # x then y
        seq = tracks[:, corner, dim]
        mask = ~np.isnan(seq)
        if not mask.any():  # never happened, but be safe
            continue
        valid_idx = np.where(mask)[0]
        valid_vals = seq[mask]
        full_idx = np.arange(N)
        seq[:] = np.interp(full_idx, valid_idx, valid_vals)

# 2) pass: draw result
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUT, fourcc, fps, (w, h))

cap = cv2.VideoCapture(VIDEO_IN)
for f in range(N):
    ok, frame = cap.read()
    if not ok:
        break

    quad = tracks[f].astype(np.int32)
    green = np.zeros_like(frame)
    cv2.fillPoly(green, [quad], (0, 255, 0))
    frame = cv2.addWeighted(frame, 1 - ALPHA, green, ALPHA, 0)
    cv2.polylines(frame, [quad], True, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("finished →", VIDEO_OUT)


finished → aruco_green_overlay.mp4
